# Fallacy Search Analysis

In [ ]:
%load_ext autoreload
%autoreload 2

from src.mafalda import get_mafalda_df
from src.constants import RESPONSE_ERROR
from src.analysis import get_sanity_check, add_search_scores
import seaborn as sns

sns.set_theme()

## Experiment 3.1: Fallacy Search

In [ ]:
filename_e31 = 'data/mafalda_e31.csv'
df_mafalda_e31 = get_mafalda_df(filename_e31)
df_mafalda_e31.head()

### Scoring and Sanity Check

In [99]:
add_search_scores(df_mafalda_e31)

In [100]:
# Check completeness of responses, predictions and scores
df_mafalda_e31.replace(['', RESPONSE_ERROR], None).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4640 entries, 0 to 4639
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   step                       4640 non-null   object  
 1   entity                     4640 non-null   object  
 2   fallacy                    4640 non-null   category
 3   label                      4640 non-null   category
 4   category                   4640 non-null   category
 5   subcategory                4640 non-null   category
 6   gpt_4o_mini_response       4640 non-null   object  
 7   claude_3_haiku_response    4640 non-null   object  
 8   gemini_1_5_flash_response  4640 non-null   object  
 9   gpt_4o_mini_pred           4640 non-null   category
 10  gpt_4o_mini_score          4640 non-null   UInt8   
 11  claude_3_haiku_pred        4640 non-null   category
 12  claude_3_haiku_score       4640 non-null   UInt8   
 13  gemini_1_5_flash_pred      4640 n

In [101]:
get_sanity_check(df_mafalda_e31)

,response_length_mean,missing_responses,invalid_predictions
gpt_4o_mini,3.4,0,0
claude_3_haiku,3.3,0,0
gemini_1_5_flash,2.4,0,0
